In [1]:
import tensorflow as tf
#import tensorflow.nn.rnn_cell as rnn
import numpy as np
#import tensorflow.keras as keras
#from tensorflow.keras.initializers import Constant
import tensorflow.keras.backend as K
from tensorflow.python.platform import gfile

from tqdm import tqdm
import os

In [2]:
data_x=np.loadtxt('./TapData/100HzTapDataValidation/PSDSData (24)_raw.csv',delimiter=",")
data_y = np.loadtxt('./TapData/100HzTapDataValidation/Tap(24)_y_100Hz.csv',delimiter=",")


In [4]:
new_x = np.empty_like(data_x[0:1])
print(new_x.shape)
new_y = np.empty_like(data_y[0:1])
for i in range(int(len(data_x)/4)):
    temp  = data_x[i*4:(i+1)*4].mean(axis=0)
    for j in range(len(temp)):
        if temp[j]>=0:
            temp[j] = data_x[i*4:(i+1)*4].max(axis=0)[j]
        else:
            temp[j] = data_x[i*4:(i+1)*4].min(axis=0)[j]
        
    new_x=np.append(new_x,[temp],axis=0)
    new_y=np.append(new_y,data_y[i*4:(i+1)*4].max())
new_x=np.delete(new_x,0,0)
new_y=np.delete(new_y,0,0)

(1, 9)


In [8]:
np.savetxt('./TapData/100HzTapDataValidation/data_25Hz_raw.csv',new_x,delimiter=",")
np.savetxt('./TapData/100HzTapDataValidation/data_25Hz_y.csv',new_y,delimiter=",")

In [32]:
model_dir='./tmp/model/real_LSTM_fullData_dynamicW_tfTrain_2Dense_7_8_92.pb'

In [33]:
sess = tf.Session()
print("load graph")
with gfile.FastGFile(model_dir,'rb') as f:
    graph_def = tf.GraphDef()
graph_def.ParseFromString(f.read())
sess.graph.as_default()
tf.import_graph_def(graph_def, name='')
graph_nodes=[n for n in graph_def.node]
names = []
for t in graph_nodes:
    names.append(t.name)
print(names)
print('graph loaded.')

load graph


W0801 15:20:42.790817  5260 deprecation.py:323] From <ipython-input-33-d69058e85418>:3: FastGFile.__init__ (from tensorflow.python.platform.gfile) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.gfile.GFile.


['input_tensor', 'input_state', 'wf', 'wf/read', 'wc', 'wc/read', 'wo', 'wo/read', 'uf', 'uf/read', 'uc', 'uc/read', 'uo', 'uo/read', 'bf', 'bf/read', 'bc', 'bc/read', 'bo', 'bo/read', 'weights1', 'weights1/read', 'weights2', 'weights2/read', 'bias1', 'bias1/read', 'bias2', 'bias2/read', 'split/split_dim', 'split', 'MatMul_1', 'MatMul_2', 'MatMul_3', 'Add_3', 'MatMul_5', 'Add_4', 'mul_1/x', 'mul_1', 'add_5/y', 'add_5', 'Const_3', 'Const_4', 'clip_by_value_1/Minimum', 'clip_by_value_1', 'Mul_2', 'Add_6', 'MatMul_6', 'Add_7', 'Tanh', 'output_c', 'Add_8', 'MatMul_7', 'Add_9', 'mul_3/x', 'mul_3', 'add_10/y', 'add_10', 'Const_5', 'Const_6', 'clip_by_value_2/Minimum', 'clip_by_value_2', 'Tanh_1', 'output_h', 'MatMul_8', 'Add_11', 'Relu', 'MatMul_9', 'output_forRelu', 'output', 'output_state/axis', 'output_state']
graph loaded.


In [49]:
Dx = new_x[:,(6-3):6].reshape((len(new_x),3))

In [50]:
num_units=64
data_state = np.zeros([1,num_units*2],dtype='float')
predictY=np.zeros([len(Dx)])

for i in tqdm(range(len(Dx))):
    feed={'input_tensor:0':Dx[i:i+1],
          'input_state:0':data_state,
            #y:data_y[i:i+1],
          #loss_weight_in:loss_weight[i:i+1]
           }

    data_state,_predictY = sess.run(['output_state:0','output:0'],feed_dict=feed)
    #print(data_h.shape)
    #data_c = sess.run(c,feed_dict=feed)
    predictY[i]=_predictY[0]

100%|██████████| 128036/128036 [00:33<00:00, 3869.90it/s]


In [51]:
np.savetxt('TF_predictY_25hz_shuffle.csv',predictY)